# Summary
### Option 1:
1. We filter the certainty of the built class to be greater than 0.62 of Google Dynamic World (GDW).
2. We also mask resulting GDW layer with a rock mask, which we manually drew from false positives from the Google Dynamic World built class.
3. We import Google Open Buildings, 2022 (GOB), convert it to raster and combine the GDW.
4. Splitting up Africa in blocks, we then export the resulting combined built layer to Google Cloud Storage.

### Option 2:
1. We import Google Open Buildings, 2022 (GOB), convert it to raster and combine the GDW.
2. For the missing countries and provinces, we use Global Human Settlements
2. Splitting up Africa in blocks, we then export the resulting combined built layer to Google Cloud Storage.

### Future
- Currently when combining Global Human Settlements with Google Open Buildings it only takes the extent of the Settlements

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

# Option 1: before 2023

In [ ]:
# Import rockMask from GoogleCloud
fs = gcsfs.GCSFileSystem(project='nature-watch-387210')

with fs.open('gs://nature-watch-bucket/vector/built_rock_mask.geojson', 'r') as f:
    data = json.load(f)

rocks = geemap.geojson_to_ee(data)

In [ ]:
# Specify dates
year = 2021
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)

In [ ]:
# Prepare layers

# Google Dynamic World
best_people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.62).selfMask()

# Use the rock mask
raster_mask = ee.Image.constant(1).paint(rocks, 0)
people_rock_masked = best_people.mask(raster_mask).unmask(0).selfMask()

# Google Open Buildings
# 2023: v3
# 2022: v2: confidence >= 0.70
# 2021: v1: confidence >= 0.80

buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v1/polygons').filter('confidence >= 0.80');

buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

# Join with other layers
built = people_rock_masked.unmask(0).add(buildings_raster.unmask(0)).gt(0).selfMask()


# Option 2: 2023
Combining Open Buildings V3 with [Global Human Settlements](https://gee-community-catalog.org/projects/ghsl/) of 2020 for missing countries.

In [ ]:
# Prepare layers

# Google Open Buildings
# 2023: v3
# 2022: v2: confidence >= 0.70
# 2021: v1: confidence >= 0.80

buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons').filter('confidence >= 0.80');

buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

# Import country borders and filter
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
missing_countries = ['Western Sahara', 'Morocco', 'Mali', 'Libya', 'Chad']
filtered_countries = countries.filter(ee.Filter.inList('ADM0_NAME', ee.List(missing_countries)))

provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")
missing_provinces = ['Cabo Delgado', 'Diffa']
filtered_provinces = provinces.filter(ee.Filter.inList('ADM1_NAME', ee.List(missing_provinces)))

filtered_boundaries = filtered_countries.merge(filtered_provinces)

# Import population dynamics
GHS_POP = ee.Image("projects/sat-io/open-datasets/GHS/GHS_POP/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0").clip(filtered_boundaries.geometry()).gt(0).selfMask();





In [ ]:
# Join with other layers
built_collection = ee.ImageCollection([buildings_raster.unmask(0).rename('b1'), GHS_POP.unmask(0)])
built = built_collection.sum().gt(0).selfMask()


# Export

In [ ]:
# Loop through blocks and export
for id, block in enumerate(ee_blocks):
  if id != 13:
    image_name = 'built2023_' + str(id)
    fileNamePrefix = 'COGS/built/2023/' + image_name


    exportConfig = {
        'image': built,
        'description': image_name,
        'bucket': 'nature-watch-bucket',
        'fileNamePrefix': fileNamePrefix,
        'scale': 30,
        'maxPixels': 3867395000,
        'region': block,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    task = ee.batch.Export.image.toCloudStorage(**exportConfig)
    task.start()

In [ ]:
# Or just export one block
id = 19
block = ee_blocks[id]


image_name = 'built2023_' + str(id)
fileNamePrefix = 'COGS/built/2023/' + image_name


exportConfig = {
    'image': built,
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 30,
    'maxPixels': 3867395000,
    'region': block,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

In [ ]:
buildings2 = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons')

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(buildings2, {}, 'buildings2')

Map.setCenter(31.47, -24.98, 12)
Map